# 資料前處理(Label encoding、 One hot encoding)
這兩個編碼方式的目的是為了將類別 (categorical)或是文字(text)的資料轉換成數字，而讓程式能夠更好的去理解及運算。
> Label encoding : 把每個類別 mapping 到某個整數，不會增加新欄位

> One hot encoding : 為每個類別新增一個欄位，用 0/1 表示是否

![](images/Encoder.PNG)


## Encoding Categorical features (or label)
![](images/Encoding.PNG)


In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});
df

,blood,Y,Z
0,A,high,NaN
1,B,low,NaN
2,AB,high,-1196.0
3,O,mid,72.0
4,B,mid,83.0


# 方法一：sklearn - label encoder + onehot encoder
>onehot encoder要用2D array，若維度所以要用reshape(-1,1)<br>
>onehot encoder要數字，若資料文文字要先用label encoder轉數字

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(df['blood'])
print(encoded_Y)
df['blood'] = encoded_Y
df

[0 2 1 3 2]


,blood,Y,Z
0,0,high,NaN
1,2,low,NaN
2,1,high,-1196.0
3,3,mid,72.0
4,2,mid,83.0


In [8]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
d = np.array(df['blood'])
d.shape
onehot_df = onehot.fit_transform(d.reshape(-1,1)).toarray()  #d.reshape(-1,1)：將一維陣列重塑為二維陣列（每個元素一行）
onehot_df

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

## One hot encoding
One Hot encoding的編碼邏輯為將類別拆成多個行(column)，每個列中的數值由1、0替代，當某一列的資料存在的該行的類別則顯示1，反則顯示0。

然在指定column進行編碼的情形下，One hot encoding<b>無法直接對字串進行編碼，必須先透過Label encoding將字串以數字取代後再進行One hot encoding處理。</b>

> categorical_features = [0]: 表示欲在data上執行One hot encoding的index為0

> data_le: 為經過Label encoding編碼的資料(註:OneHotEncoder的輸入要為2-D array，而Label encoding為1-D array)


OneHotEncoder會轉出scipy.csr_matrix資料結構用.toarray()轉array
從結果可以知道，數字0的column 代表的是A、數字1的column 代表的是B，而數字2的column 代表的是AB。
除了轉換字串外，One hot encoding也可以轉換數字。在此處的data就不需要先經過Label encoding編碼

```python
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
data = np.array(columnTransformer.fit_transform(data), dtype = str) 
```

In [9]:
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 

data = np.array(columnTransformer.fit_transform(df), dtype = str)
data

data_le = pd.DataFrame(data)
data_le

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,high,nan
1,0.0,0.0,1.0,0.0,low,nan
2,0.0,1.0,0.0,0.0,high,-1196.0
3,0.0,0.0,0.0,1.0,mid,72.0
4,0.0,0.0,1.0,0.0,mid,83.0


In [9]:
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 

data = np.array(columnTransformer.fit_transform(df), dtype = str)
data

data_le = pd.DataFrame(data)
data_le

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,high,nan
1,0.0,0.0,1.0,0.0,low,nan
2,0.0,1.0,0.0,0.0,high,-1196.0
3,0.0,0.0,0.0,1.0,mid,72.0
4,0.0,0.0,1.0,0.0,mid,83.0


In [27]:
!pip install tensorflow

In [30]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});

# label encoder 
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(df['blood'])
print(encoded_Y)
df['blood'] = encoded_Y
df

# convert integers to one hot encoding
keras_onehot = to_categorical(encoded_Y)
keras_onehot

[0 2 1 3 2]


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

# 方法二：Keras - label encoder + to_categorical
>to_categorical要數字，若資料文文字要先用label encoder轉數字

## 方法三：pd.get_dummies方法
![](images/Encoding_pd.PNG)
pd.get_dummies(df)
>get_dummies可以直接轉字串，反而無法轉換數字<br>
>get_dummies沒指定columns，會全部轉換

In [21]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]})
df1 = pd.get_dummies(df)
print(df1)
df2 = pd.get_dummies(df.blood)
print(df2)

        Z  blood_A  blood_AB  blood_B  blood_O  Y_high  Y_low  Y_mid
0     NaN     True     False    False    False    True  False  False
1     NaN    False     False     True    False   False   True  False
2 -1196.0    False      True    False    False    True  False  False
3    72.0    False     False    False     True   False  False   True
4    83.0    False     False     True    False   False  False   True
       A     AB      B      O
0   True  False  False  False
1  False  False   True  False
2  False   True  False  False
3  False  False  False   True
4  False  False   True  False


## 練習一：sklearn - label encoder + onehot encoder
下面的資料可以看到country那欄皆為字串， 大部分的模型都是基於數學運算，字串無法套入數學模型進行運算，<br>
在此先對其進行Label encoding編碼，我們從 sklearn library中導入 LabelEncoder class，對第一行資料進行fit及transform並取代之。

In [41]:
import numpy as np
import pandas as pd
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

# Step 1: Label Encoder
label_encoder = LabelEncoder()
encoded_country = label_encoder.fit_transform(data['Country'])
print("\nLabel Encoder 結果：")
print(encoded_country)
print("對應關係：", dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))

# Step 2: OneHot Encoder
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_result = onehot_encoder.fit_transform(encoded_country.reshape(-1, 1))
print("\nOneHot Encoder 結果：")
print(onehot_result)

# Step 3: 合併結果成完整 DataFrame
country_names = label_encoder.classes_
onehot_df = pd.DataFrame(onehot_result, columns=country_names)
onehot_df




Label Encoder 結果：
[2 0 1 0 1 2]
對應關係： {'Australia': 0, 'Ireland': 1, 'Taiwan': 2}

OneHot Encoder 結果：
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


,Australia,Ireland,Taiwan
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
5,0.0,0.0,1.0


## 練習二：Keras - label encoder + to_categorical

In [37]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

# Step 1: Label Encoder
label_encoder = LabelEncoder()
encoded_country = label_encoder.fit_transform(data['Country'])

# Step 2: to_categorical (使用 TensorFlow Keras 的 to_categorical)
categorical_result = to_categorical(encoded_country)
print("\nto_categorical 結果（陣列格式）：")
print(categorical_result)



to_categorical 結果（陣列格式）：
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 練習三：Pandas.get_dummies
>　get_dummies : 僅能將字串轉換為One hot encoding表示形式， 沒指定columns會全部轉換。

In [43]:
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

# get_dummies：僅將字串轉換為 One hot encoding 表示形式，沒指定 columns 會全部轉換
dummies_result = pd.get_dummies(data)
print(dummies_result)

country_dummies = pd.get_dummies(data['Country'])
print(country_dummies)

   Age  Salary  Country_Australia  Country_Ireland  Country_Taiwan
0   25   20000              False            False            True
1   30   32000               True            False           False
2   45   59000              False             True           False
3   35   60000               True            False           False
4   22   43000              False             True           False
5   36   52000              False            False            True
   Australia  Ireland  Taiwan
0      False    False    True
1       True    False   False
2      False     True   False
3       True    False   False
4      False     True   False
5      False    False    True
